In [1]:
# Install gdown if not already present
!pip install -q gdown

# Reinstall libraries with versions compatible with Google Colab
!pip install --upgrade --quiet \
    numpy==1.26.4 \
    pandas==2.2.2 \
    scikit-learn==1.3.2 \
    xgboost==2.0.3 \
    lightgbm==4.1.0 \
    catboost==1.2.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [2]:
# Reinstall NumPy first
!pip install --force-reinstall --no-cache-dir numpy==1.26.4

# Now reinstall CatBoost so it compiles with the correct NumPy ABI
!pip install --force-reinstall --no-cache-dir catboost==1.2.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 201.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 242.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 kB 273.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 176.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [5]:
from google.colab import files
uploaded = files.upload()

Saving IntGenbyFuel2024.xlsx to IntGenbyFuel2024 (1).xlsx
Saving IntGenbyFuel2022.xlsx to IntGenbyFuel2022 (1).xlsx
Saving IntGenbyFuel2023.xlsx to IntGenbyFuel2023 (1).xlsx
Saving rpt.00013061.0000000000000000.RTMLZHBSPP_2024.xlsx to rpt.00013061.0000000000000000.RTMLZHBSPP_2024.xlsx
Saving rpt.00013061.0000000000000000.RTMLZHBSPP_2023.xlsx to rpt.00013061.0000000000000000.RTMLZHBSPP_2023.xlsx
Saving rpt.00013061.0000000000000000.RTMLZHBSPP_2022.xlsx to rpt.00013061.0000000000000000.RTMLZHBSPP_2022.xlsx


In [2]:
pip install pandas openpyxl


In [6]:
import pandas as pd

def process_year(year):
    print(f"\n🔄 Processing year: {year}")

    # === Step 1: Load IntGenbyFuel ===
    intgen_file = f"IntGenbyFuel{year}.xlsx"
    intgen_xls = pd.ExcelFile(intgen_file)
    fuel_sheets = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    fuels = ['Coal', 'Biomass', 'Gas', 'Nuclear']
    all_gen_data = []

    for sheet in fuel_sheets:
        print(f"  • IntGen sheet: {sheet}")
        df = intgen_xls.parse(sheet)
        df = df[df['Fuel'].isin(fuels)].copy()

        time_cols = [col for col in df.columns if ':' in str(col)]
        df_melted = df.melt(id_vars=['Date', 'Fuel'], value_vars=time_cols,
                            var_name='Time', value_name='MW')
        df_melted['Timestamp'] = pd.to_datetime(df_melted['Date'].astype(str) + ' ' + df_melted['Time'], errors='coerce')
        df_melted.dropna(subset=['Timestamp'], inplace=True)
        df_melted['Hour'] = df_melted['Timestamp'].dt.floor('h')
        hourly_df = df_melted.groupby(['Hour', 'Fuel'])['MW'].sum().reset_index()
        all_gen_data.append(hourly_df)

    gen_df = pd.concat(all_gen_data, ignore_index=True)

    # === Step 2: Load RTM price data ===
    rtm_file = f"rpt.00013061.0000000000000000.RTMLZHBSPP_{year}.xlsx"
    rtm_xls = pd.ExcelFile(rtm_file)
    all_rtm_data = []

    for sheet in fuel_sheets:
        print(f"  • RTM sheet: {sheet}")
        df = rtm_xls.parse(sheet, usecols=['Delivery Date', 'Delivery Hour', 'Settlement Point Name', 'Settlement Point Price'])
        df = df[df['Settlement Point Name'] == 'HB_BUSAVG'].copy()
        df['Delivery Hour'] = df['Delivery Hour'].astype(int)
        df['Hour'] = pd.to_datetime(df['Delivery Date']) + pd.to_timedelta(df['Delivery Hour'], unit='h')
        all_rtm_data.append(df[['Hour', 'Settlement Point Price']])

    rtm_df = pd.concat(all_rtm_data, ignore_index=True)
    rtm_hourly = rtm_df.groupby('Hour')['Settlement Point Price'].mean().reset_index()
    rtm_hourly.columns = ['Hour', 'Price ($/MWh)']

    # === Step 3: Merge & Save ===
    merged = pd.merge(gen_df, rtm_hourly, on='Hour', how='inner')
    output_file = f"Full_{year}_Hourly_Gen_and_Price.csv"
    merged.to_csv(output_file, index=False)

    print(f"✅ Saved: {output_file}")


# === Process multiple years ===
for year in [2022, 2023, 2024]:
    process_year(year)



🔄 Processing year: 2022
  • IntGen sheet: Jan
  • IntGen sheet: Feb
  • IntGen sheet: Mar
  • IntGen sheet: Apr
  • IntGen sheet: May
  • IntGen sheet: Jun
  • IntGen sheet: Jul
  • IntGen sheet: Aug
  • IntGen sheet: Sep
  • IntGen sheet: Oct
  • IntGen sheet: Nov
  • IntGen sheet: Dec
  • RTM sheet: Jan
  • RTM sheet: Feb
  • RTM sheet: Mar
  • RTM sheet: Apr
  • RTM sheet: May
  • RTM sheet: Jun
  • RTM sheet: Jul
  • RTM sheet: Aug
  • RTM sheet: Sep
  • RTM sheet: Oct
  • RTM sheet: Nov
  • RTM sheet: Dec
✅ Saved: Full_2022_Hourly_Gen_and_Price.csv

🔄 Processing year: 2023
  • IntGen sheet: Jan
  • IntGen sheet: Feb
  • IntGen sheet: Mar
  • IntGen sheet: Apr
  • IntGen sheet: May
  • IntGen sheet: Jun
  • IntGen sheet: Jul
  • IntGen sheet: Aug
  • IntGen sheet: Sep
  • IntGen sheet: Oct
  • IntGen sheet: Nov
  • IntGen sheet: Dec
  • RTM sheet: Jan
  • RTM sheet: Feb
  • RTM sheet: Mar
  • RTM sheet: Apr
  • RTM sheet: May
  • RTM sheet: Jun
  • RTM sheet: Jul
  • RTM sheet: Au

In [8]:
pd.read_csv('Full_2024_Hourly_Gen_and_Price.csv')




,Hour,Fuel,MW,Price ($/MWh)
0,2024-01-01 01:00:00,Biomass,12.866443,14.0450
1,2024-01-01 01:00:00,Coal,5895.822686,14.0450
2,2024-01-01 01:00:00,Gas,1016.704588,14.0450
3,2024-01-01 01:00:00,Nuclear,5100.049072,14.0450
4,2024-01-01 02:00:00,Biomass,10.415912,15.9700
...,...,...,...,...
35123,2024-12-31 22:00:00,Nuclear,5085.549549,22.6250
35124,2024-12-31 23:00:00,Biomass,12.204715,22.2325
35125,2024-12-31 23:00:00,Coal,7419.532962,22.2325
35126,2024-12-31 23:00:00,Gas,1080.756969,22.2325


In [9]:
pd.read_csv('Full_2023_Hourly_Gen_and_Price.csv')


,Hour,Fuel,MW,Price ($/MWh)
0,2022-01-01 01:00:00,Biomass,22.297569,62.3975
1,2022-01-01 01:00:00,Coal,10269.999131,62.3975
2,2022-01-01 01:00:00,Gas,1116.244747,62.3975
3,2022-01-01 01:00:00,Nuclear,5078.955677,62.3975
4,2022-01-01 02:00:00,Biomass,22.300089,31.6825
...,...,...,...,...
35027,2022-12-31 22:00:00,Nuclear,5109.800227,2.4500
35028,2022-12-31 23:00:00,Biomass,16.738028,-1.6325
35029,2022-12-31 23:00:00,Coal,4080.609499,-1.6325
35030,2022-12-31 23:00:00,Gas,658.131392,-1.6325


In [10]:
pd.read_csv('Full_2022_Hourly_Gen_and_Price.csv')

,Hour,Fuel,MW,Price ($/MWh)
0,2022-01-01 01:00:00,Biomass,22.297569,62.3975
1,2022-01-01 01:00:00,Coal,10269.999131,62.3975
2,2022-01-01 01:00:00,Gas,1116.244747,62.3975
3,2022-01-01 01:00:00,Nuclear,5078.955677,62.3975
4,2022-01-01 02:00:00,Biomass,22.300089,31.6825
...,...,...,...,...
35027,2022-12-31 22:00:00,Nuclear,5109.800227,2.4500
35028,2022-12-31 23:00:00,Biomass,16.738028,-1.6325
35029,2022-12-31 23:00:00,Coal,4080.609499,-1.6325
35030,2022-12-31 23:00:00,Gas,658.131392,-1.6325


In [14]:
import pandas as pd


file_paths = [
    "Full_2022_Hourly_Gen_and_Price.csv",
    "Full_2023_Hourly_Gen_and_Price.csv",
    "Full_2024_Hourly_Gen_and_Price.csv"
]


all_data = []


for file_path in file_paths:
    df = pd.read_csv(file_path)
    all_data.append(df)


combined_df = pd.concat(all_data, ignore_index=True)


combined_file = "Combined_Hourly_Gen_and_Price.csv"
combined_df.to_csv(combined_file, index=False)

print(f" Combined file saved as: {combined_file}")


 Combined file saved as: Combined_Hourly_Gen_and_Price.csv


In [15]:
pd.read_csv('Combined_Hourly_Gen_and_Price.csv')

,Hour,Fuel,MW,Price ($/MWh)
0,2022-01-01 01:00:00,Biomass,22.297569,62.3975
1,2022-01-01 01:00:00,Coal,10269.999131,62.3975
2,2022-01-01 01:00:00,Gas,1116.244747,62.3975
3,2022-01-01 01:00:00,Nuclear,5078.955677,62.3975
4,2022-01-01 02:00:00,Biomass,22.300089,31.6825
...,...,...,...,...
105187,2024-12-31 22:00:00,Nuclear,5085.549549,22.6250
105188,2024-12-31 23:00:00,Biomass,12.204715,22.2325
105189,2024-12-31 23:00:00,Coal,7419.532962,22.2325
105190,2024-12-31 23:00:00,Gas,1080.756969,22.2325


In [16]:



combined_file = "Combined_Hourly_Gen_and_Price.csv"
df = pd.read_csv(combined_file)


if 'Price ($/MWh)' in df.columns:
    df.drop('Price ($/MWh)', axis=1, inplace=True)


output_file = "Combined_Hourly_Gen_Without_Price.csv"
df.to_csv(output_file, index=False)

print(f" File without 'Price' column saved as: {output_file}")

 File without 'Price' column saved as: Combined_Hourly_Gen_Without_Price.csv


In [17]:
pd.read_csv('Combined_Hourly_Gen_Without_Price.csv')

,Hour,Fuel,MW
0,2022-01-01 01:00:00,Biomass,22.297569
1,2022-01-01 01:00:00,Coal,10269.999131
2,2022-01-01 01:00:00,Gas,1116.244747
3,2022-01-01 01:00:00,Nuclear,5078.955677
4,2022-01-01 02:00:00,Biomass,22.300089
...,...,...,...
105187,2024-12-31 22:00:00,Nuclear,5085.549549
105188,2024-12-31 23:00:00,Biomass,12.204715
105189,2024-12-31 23:00:00,Coal,7419.532962
105190,2024-12-31 23:00:00,Gas,1080.756969


In [18]:
from google.colab import files

# To download the file
files.download('Combined_Hourly_Gen_Without_Price.csv')
files.download('Combined_Hourly_Gen_and_Price.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>